In [1]:
import pandas as pd
import requests
from datetime import datetime
import json
import os
from dateutil import parser

The following code helps to determine whether new data is inserted in api database or not, if so then we have to pull new movies data and put that in database and update tracking parameters. 

In [4]:
last_page =  requests.get('https://yts.torrentbay.to/api/v2/list_movies.json?page=821&limit=50').json()
first_page = requests.get('https://yts.torrentbay.to/api/v2/list_movies.json?page=2&limit=50').json()


In [3]:
first_page

{'status': 'ok',
 'status_message': 'Query was successful',
 'data': {'movie_count': 41052,
  'limit': 50,
  'page_number': 1,
  'movies': [{'id': 41853,
    'url': 'https://yts.torrentbay.to/movies/a-thousand-little-cuts-2022',
    'imdb_code': 'tt11244558',
    'title': 'A Thousand Little Cuts',
    'title_english': 'A Thousand Little Cuts',
    'title_long': 'A Thousand Little Cuts (2022)',
    'slug': 'a-thousand-little-cuts-2022',
    'year': 2022,
    'rating': 0,
    'runtime': 87,
    'genres': ['Thriller'],
    'summary': 'A young woman wakes up in a hospital, convinced she is recovering from minor surgery only to find herself in a battle of wits with a psychiatrist who can&#39;t let her leave until she remembers &quot;what happened that night.&quot; Events turn a darker corner as doctor and patient try to unlock not only what traumatic event Anne is suppressing, but also who was there, why it happened, and why Anne&#39;s subconscious is fighting so hard to prevent her from ta

In [6]:
last_uploaded_date = parser.parse('2022-04-28')

In [7]:
count = 0
for movie in first_page['data']['movies']:
    if last_uploaded_date < parser.parse(movie['torrents'][0]['date_uploaded'].split(' ')[0]):
        count +=1
        print(movie['title'])

Love, Life and Goldfish
Monkey Enters Lanka
Siembra
Les Rose
Disorientation Is Not a Crime
Snipers
Accused of Murder
Brad Williams: Fun Size
Honokaa Boy
American Friends
Collective: Unconscious
Adventures of a Private Eye
Trophy Wife
Due Season
Circus of the Dead
Vietnam, Texas
Bright Road
I Don't Have Any Money Left in My Retirement Account
The Killing Game
Prayers for the Stolen
Going to Hell: The Movie
Prelude to a Kiss
The Heroin Busters
The Big Racket
Honeymoon with My Mother
Rumspringa
The Aviary
Marvelous and the Black Hole
Crush
Unplugging
White Rhino
In the Wake
BBC Sunday-Night Theatre Nineteen Eighty-Four
Dr. Strange
Look in Any Window


In [8]:
count

35

{'imDbId': 'tt1680045',
 'title': '360',
 'fullTitle': '360 (2011)',
 'type': 'Movie',
 'year': '2011',
 'directors': {'job': 'Director',
  'items': [{'id': 'nm0576987',
    'name': 'Fernando Meirelles',
    'description': ''}]},
 'writers': {'job': 'Writer',
  'items': [{'id': 'nm0604948',
    'name': 'Peter Morgan',
    'description': '(screenplay)'},
   {'id': 'nm0774217',
    'name': 'Arthur Schnitzler',
    'description': '(play) (uncredited)'}]},
 'actors': [{'id': 'nm2521724',
   'image': 'https://imdb-api.com/images/original/MV5BMTY2MjExMzcwMV5BMl5BanBnXkFtZTcwMDMzMTE5Ng@@._V1_Ratio0.7273_AL_.jpg',
   'name': 'Lucia Siposová',
   'asCharacter': 'Mirka'},
  {'id': 'nm4387549',
   'image': 'https://imdb-api.com/images/original/MV5BMTg3MzU0MDQ1Ml5BMl5BanBnXkFtZTcwMzMzMTE5Ng@@._V1_Ratio0.7273_AL_.jpg',
   'name': 'Gabriela Marcinková',
   'asCharacter': 'Anna'},
  {'id': 'nm0471406',
   'image': 'https://imdb-api.com/images/original/MV5BMjEwNTEwNTUyOV5BMl5BanBnXkFtZTgwNDI1NDkzMDI@.